# AEWS Python Notebook 09e: complete (pre-batch) WQ algorithm

**Author**: Eric Lehmann, CSIRO Data61  
**Date**: Aug. 26, 2016.

**Note**: this notebook should be accessible and viewable at https://github.com/eric542/agdc_v2/tree/master/notebooks.

## Summary

This notebook brings together all the AEWS components tested / implemented in earlier notebooks. It implements the whole AEWS workflow, starting from the selected ROI and AGDC data, and ultimately creating a NetCDF dataset containing a time series of WQ maps with associated ancillary information. For testing purposes, this is here carried out for a single selected polygon, and thus represents a pre-batch version. A fully automated batch script executed on the NCI will automatically iterate through all the polygons of interest.

This notebook version (09e) started off as a copy of '_AEWS Python Notebook 09d_', updated to parallelise the main waterbody-processing loop.

Main changes from previous version include:

+ re-definition of lakes processing loop as function
+ changes to the handling of API connections / calls to avoid recurring connections to the database
+ attempt to reduce potential race conditions from multiple threads when reading from `.shp` file


## Preliminaries

This (Jupyter) notebook was written for use on the NCI's VDI system, with the following pre-loaded module:

```
 $ module use /g/data/v10/public/modules/modulefiles --append
 $ module load agdc-py2-prod 
```

In [1]:
import numpy as np
import ogr, osr, gdal
import xarray as xr
import os
import multiprocessing as mp

from netCDF4 import Dataset, num2date, date2num
from datetime import timedelta, datetime    # date

from matplotlib.path import Path   # for point-in-polygon

from scipy import ndimage
from scipy.spatial.distance import pdist

import time as timer
from pprint import pprint
from __future__ import print_function

import datacube.api
from datacube.analytics.analytics_engine import AnalyticsEngine
from datacube.execution.execution_engine import ExecutionEngine

from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
from matplotlib import rcParams as rcp

from sklearn import linear_model
import pickle

In [2]:
def expand_mask(mask_arr, npix):
    # Uses the True/False (masked/non-masked) values in the array 'mask_arr' and 
    # expands the True values spatially by 'npix' pixels. The value 'npix' can be
    # non-integer, i.e. the mask can be expanded by any spatial distance.
    nmid = np.floor(npix)
    nmax = int( nmid*2 + 1 )
    struc = np.zeros((nmax, nmax), dtype='bool')
    for ii in range(nmax):   # create desired binary structure for morphological operation
        for jj in range(ii,nmax):
            if pdist( [[nmid,nmid], [ii,jj]] ) <= npix:
                struc[ii,jj] = True
                struc[jj,ii] = True
    return ndimage.binary_dilation(mask_arr, structure=struc)


def merge_dup_dates(xrda):
    # Takes in an xarray.DataArray 'xrda' as input and merges 
    # datasets (time slices) within it that have the same dates.
    # Returns the modified DataArray and vector of selected dates.
    dates = xrda.coords['time'].values
    n_dates = len( dates )
    
    # Convert UTM times to local dates (d/m/y only):
    str_dates = np.zeros(n_dates).astype('str')
    for ii in range(n_dates):
        str_dates[ii] = str( dates[ii] )[:10]
    
    # Remove duplicated dates:
    rem_ind = np.zeros(n_dates).astype('bool')   # keep track of which duplicated dates to remove
    for ind in range(1,n_dates):
        dup_ind = np.where( str_dates[:ind]==str_dates[ind] )[0]   # check for duplicated date up to current index
        if len( dup_ind )!=0:   # found (at least) one duplicate
            dup_ind = dup_ind[0]   # only use the first index if multiple dates returned
            rem_ind[ind] = True    # remove current date index
            ind_n_nans = np.sum( np.isnan( xrda[ind] ) ) # nr of NaN pixels in each image
            dup_n_nans = np.sum( np.isnan( xrda[dup_ind] ) )
            if ind_n_nans==0:     # current data has no NaN's, use it instead of duplicate date (copy it to lowest index)
                xrda[dup_ind] = xrda[ind]
                xrda['time'].values[dup_ind] = xrda['time'].values[ind]
            elif dup_n_nans!=0:   # if duplicate date has no NaN's: do nothing (use it instead of current date)
                if dup_n_nans<ind_n_nans:   # duplicate date has less NaN's: fill it in with current data
                    tmp = np.where( np.isnan(xrda[dup_ind]) )
                    xrda[dup_ind].values[tmp] = xrda[ind].values[tmp]
                else:   # dup_n_nans>=ind_n_nans -- duplicate date has more NaN's: use it to fill in current data
                    tmp = np.where( np.isnan(xrda[ind]) )
                    xrda[ind].values[tmp] = xrda[dup_ind].values[tmp]
                    xrda[dup_ind] = xrda[ind]   # save results to lowest date index, in case >2 slices have same date
                    xrda['time'].values[dup_ind] = xrda['time'].values[ind]
    
    return xrda[~rem_ind]


def load_wofs_data(wofs_path, min_lon, max_lon, min_lat, max_lat):
    # Returns array of WOFS data for given extents. Can read data from 
    # multiple adjacent tiles if necessary, at most 2 in either direction.
    lon_idx1 = np.floor(min_lon); lon_idx2 = np.floor(max_lon)
    dlon = abs(lon_idx1-lon_idx2)
    lat_idx1 = np.floor(min_lat); lat_idx2 = np.floor(max_lat)
    dlat = abs(lat_idx1-lat_idx2)
    if ( dlon>1 or dlat>1 ):
        raise RuntimeError("Lake extents span 3 or more WOFS tiles (in at least one dimension).")
    adj_tile_lon = (dlon==1)    # two adjacent 'horizontal' WOFS tiles
    adj_tile_lat = (dlat==1)    # two adjacent 'vertical' WOFS tiles

    # Load the first (top-left) of (potentially) several WOFS tiles:
    WOFS_fname = wofs_path + 'percentWater_{:3.0f}_{:04.0f}.tiff'.format( np.floor(min_lon), np.floor(max_lat) )
    WOFS_dataset = gdal.Open(WOFS_fname)
    if WOFS_dataset==None:
        raise RuntimeError( "Cannot open WOFS data file '{}'.".format(WOFS_fname) )
    WOFS_array = WOFS_dataset.ReadAsArray()
    WOFS_geotx = list( WOFS_dataset.GetGeoTransform() )
    WOFS_proj = WOFS_dataset.GetProjection()   # geodetic lat/lon
    # WOFS_srs = osr.SpatialReference( wkt=WOFS_proj )
    WOFS_lonvec = np.arange(WOFS_array.shape[0]) * WOFS_geotx[1] + WOFS_geotx[0]
    WOFS_latvec = np.arange(WOFS_array.shape[1]) * WOFS_geotx[5] + WOFS_geotx[3]
    WOFS_dataset = None   # closes the gdal dataset

    if ( adj_tile_lon ):   # if 2 horizontal tiles, load (top) right tile
        WOFS_fname = wofs_path + 'percentWater_{:3.0f}_{:04.0f}.tiff'.format( np.floor(max_lon), np.floor(max_lat) )
        WOFS_dataset = gdal.Open(WOFS_fname)
        if WOFS_dataset==None:
            raise RuntimeError( "Cannot open WOFS data file '{}'.".format(WOFS_fname) )
        wa2 = WOFS_dataset.ReadAsArray()
        wg2 = list( WOFS_dataset.GetGeoTransform() )
        WOFS_lonvec = np.concatenate(( WOFS_lonvec, np.arange(wa2.shape[0]) * wg2[1] + wg2[0] ),0)
        WOFS_array = np.concatenate((WOFS_array,wa2),1)   # column bind
        WOFS_dataset = None   # closes the gdal dataset

    if ( adj_tile_lat ):   # if 2 vertical tiles: load bottom (left) tile
        WOFS_fname = wofs_path + 'percentWater_{:3.0f}_{:04.0f}.tiff'.format( np.floor(min_lon), np.floor(min_lat) )
        WOFS_dataset = gdal.Open(WOFS_fname)
        if WOFS_dataset==None:
            raise RuntimeError( "Cannot open WOFS data file '{}'.".format(WOFS_fname) )
        wa2 = WOFS_dataset.ReadAsArray()
        wg2 = list( WOFS_dataset.GetGeoTransform() )
        WOFS_latvec = np.concatenate(( WOFS_latvec, np.arange(wa2.shape[1]) * wg2[5] + wg2[3] ),0)
        WOFS_dataset = None   # closes the gdal dataset

        if ( adj_tile_lon ):   # if 2 horizontal & 2 vertical tiles: load bottom right tile
            WOFS_fname = wofs_path + 'percentWater_{:3.0f}_{:04.0f}.tiff'.format( np.floor(max_lon), np.floor(min_lat) )
            WOFS_dataset = gdal.Open(WOFS_fname)
            if WOFS_dataset==None:
                raise RuntimeError( "Cannot open WOFS data file '{}'.".format(WOFS_fname) )
            wa3 = WOFS_dataset.ReadAsArray()
            wa2 = np.concatenate((wa2,wa3),1)   # column bind
            WOFS_dataset = None   # closes the gdal dataset

        WOFS_array = np.concatenate((WOFS_array,wa2),0)   # row bind
    
    # Extract WOFS over region of interest:
    yind_min = np.where( WOFS_lonvec>=min_lon )[0][0]
    yind_max = np.where( WOFS_lonvec<=max_lon )[0][-1]
    WOFS_lonvec = WOFS_lonvec[yind_min:yind_max+1]
    xind_max = np.where( WOFS_latvec>=min_lat )[0][-1]
    xind_min = np.where( WOFS_latvec<=max_lat )[0][0]
    WOFS_latvec = WOFS_latvec[xind_min:xind_max+1]
    WOFS_array = WOFS_array[xind_min:xind_max+1, yind_min:yind_max+1]
    WOFS_geotx[0] = WOFS_geotx[0] + yind_min*WOFS_geotx[1]
    WOFS_geotx[3] = WOFS_geotx[3] + xind_min*WOFS_geotx[5]

    return WOFS_array, WOFS_proj, WOFS_geotx, WOFS_lonvec, WOFS_latvec


def solar_day(utc, lon):
    # returns approx. solar data from 'utc' time and average 'lon'.
    # Based on GA's implementation, available at:
    # https://github.com/data-cube/agdc-v2/blob/38a3430b3f49977f3b07355ba0d5d2d4ad7bf965/datacube/api/geo_xarray.py
    seconds_per_degree = 240
    offset_seconds = int(lon * seconds_per_degree)
    offset = np.timedelta64(offset_seconds, 's')
    return utc + offset


def write_to_log(logf,strn,print_screen=True):
    # Prints info 'strn' to command line as well as log file 'logf'
    logf.write( strn + '\n')
    if print_screen and not no_screen_outputs:
        print( strn )

In [3]:
def ftr_loop(ftr):
    # Function implementing the processing of waterbody 'ftr'.
    # 'Returns' a list of lists: [out_log_str, out_cpu_arr, out_nc_flag]
    
    # loop 'outputs':
    out_log_str = ''
    out_nc_flag = False
    out_cpu_arr = np.ones((1,6))*np.nan

    poly_t0 = timer.time()
    
    # Log info:
    out_log_str += "Processing polygon nr. {} of {} ...".format(ftr+1,n_lakes) + '\n'
    out_log_str += "  Polygon name: {}".format(lakes_dispname_list[ftr]) + '\n'
    out_log_str += "  Surface area is {} ha.".format(round(shape_list[ftr]/10000.0,2)) + '\n'

    
    ###########################################################################
    ### Iteration over polygons: ROI polygon coordinates and extents
    ###########################################################################

    # Read in polygon coordinates: reading from .shp file from multiple parallel threads 
    # appears to lead to race conditions at times; the code below tries to mitigate this.
    np.random.seed(ftr)
    
    lake_ftr = lakes_lyr.GetFeature( lakes_ftr_idx[ftr] )
    
    tmp = lake_ftr.GetGeometryRef()
    cnt = 0
    while tmp==None:
        if cnt==0:
            out_log_str += "  Problem accessing lat/lon coords for this polygon (lake_ftr.GetGeometryRef is None)." + '\n'
            return [out_log_str, out_cpu_arr, out_nc_flag]
        cnt -= 1
        timer.sleep(np.random.random())
        tmp = lake_ftr.GetGeometryRef()
    
    ring = tmp.GetGeometryRef(0)
    cnt = 0
    while ring==None:
        if cnt==0:
            out_log_str += "  Problem accessing lat/lon coords for this polygon (ring is None)." + '\n'
            return [out_log_str, out_cpu_arr, out_nc_flag]
        cnt -= 1
        timer.sleep(np.random.random())
        ring = tmp.GetGeometryRef(0)
    
    tmp = ring.GetPoints()
    cnt = 0
    while tmp==None:
        if cnt==0:
            out_log_str += "  Problem accessing lat/lon coords for this polygon (ring.GetPoints is None)." + '\n'
            return [out_log_str, out_cpu_arr, out_nc_flag]
        cnt -= 1
        timer.sleep(np.random.random())
        tmp = ring.GetPoints()
    
    tmp = np.array(tmp)
    if tmp.shape[0]==0 or tmp.shape[1]<2:
        out_log_str += "  Problem accessing lat/lon coords for this polygon (no coords)." + '\n'
        return [out_log_str, out_cpu_arr, out_nc_flag]
    
    poly_array_geo = tmp[:,0:2]    # lon / lat coords of polygon

    tmp1 = min(poly_array_geo[:,0]) - lake_buffer_width_geo
    min_lon = (tmp1 // pix_size_geo) * pix_size_geo

    tmp2 = max(poly_array_geo[:,0]) + lake_buffer_width_geo
    max_lon = (tmp2 // pix_size_geo) * pix_size_geo + pix_size_geo

    tmp3 = min(poly_array_geo[:,1]) - lake_buffer_width_geo
    min_lat = (tmp3 // pix_size_geo) * pix_size_geo

    tmp4 = max(poly_array_geo[:,1]) + lake_buffer_width_geo
    max_lat = (tmp4 // pix_size_geo) * pix_size_geo + pix_size_geo

    # Coordinates of resulting WQ time series:
    lon_vec = np.arange(min_lon, max_lon+pix_size_geo, pix_size_geo)
    lat_vec = np.arange(min_lat, max_lat+pix_size_geo, pix_size_geo)
    mean_lon = (max_lon+min_lon)/2.0

    # Log info:
    out_log_str += "  Polygon extents: min lon.: {} --> nearest pixel: {}".format(tmp1,min_lon) + '\n'
    out_log_str += "                   max lon.: {} --> nearest pixel: {}".format(tmp2,max_lon) + '\n'
    out_log_str += "                   min lat.: {} --> nearest pixel: {}".format(tmp3,min_lat) + '\n'
    out_log_str += "                   min lat.: {} --> nearest pixel: {}".format(tmp3,min_lat) + '\n'


    ######################################################################################
    ### Iteration over polygons: check for existing dataset, determine ROI date range
    ######################################################################################

    roi_file_str = lakes_name_list[ftr] + ".nc"
    roi_save_name = save_dir_path + roi_file_str
    roi_file_exists = os.path.isfile( roi_save_name )
    
    nodata_log_name = nodata_dir_path + lakes_name_list[ftr] + ".log"
    nodata_file_exists = os.path.isfile( nodata_log_name )

    roi_start_date = datetime( int(start_date[:4]), int(start_date[5:7]), int(start_date[8:10]) )
    
    roi_append_nc = False
    if append_to_existing_nc:
        if roi_file_exists:
            roi_append_nc = True
            
            # read file attributes:
            roi_nc_grp = Dataset(roi_save_name, mode='r')
            roi_nc_dates = roi_nc_grp.variables['time']

            # consistency checks between datasets:
            if not (roi_nc_grp.variables['lat'][:]==lat_vec[::-1]).all():
                out_log_str += "  Error appending to existing NetCDF '{}': different latitudes.".format(roi_save_name) + '\n'
                return [out_log_str, out_cpu_arr, out_nc_flag]
            if not (roi_nc_grp.variables['lon'][:]==lon_vec).all():
                out_log_str += "  Error appending to existing NetCDF '{}': different longitudes.".format(roi_save_name) + '\n'
                return [out_log_str, out_cpu_arr, out_nc_flag]
            if not getattr(roi_nc_grp,'ResultsVersion')==results_version:
                out_log_str += "  Error appending to existing NetCDF '{}': different WQ outputs version.".format(roi_save_name) + '\n'
                return [out_log_str, out_cpu_arr, out_nc_flag]
            if not getattr(roi_nc_grp,'WQtype')==WQ_type:
                out_log_str += "  Error appending to existing NetCDF '{}': different WQ type.".format(roi_save_name) + '\n'
                return [out_log_str, out_cpu_arr, out_nc_flag]
            if not roi_nc_dates.calendar==netcdf_time_calendar:
                out_log_str += "  Error appending to existing NetCDF '{}': different time calendar.".format(roi_save_name) + '\n'
                return [out_log_str, out_cpu_arr, out_nc_flag]
            if not roi_nc_dates.units==netcdf_time_units:
                out_log_str += "  Error appending to existing NetCDF '{}': different time units.".format(roi_save_name) + '\n'
                return [out_log_str, out_cpu_arr, out_nc_flag]
            if not getattr(roi_nc_grp,'Name')==lakes_name_list[ftr]:
                out_log_str += "  Error appending to existing NetCDF '{}': different lake name.".format(roi_save_name) + '\n'
                return [out_log_str, out_cpu_arr, out_nc_flag]
            if not getattr(roi_nc_grp,'DisplayName')==lakes_dispname_list[ftr]:
                out_log_str += "  Error appending to existing NetCDF '{}': different display name.".format(roi_save_name) + '\n'
                return [out_log_str, out_cpu_arr, out_nc_flag]
            if not getattr(roi_nc_grp,'EPSG')==geo_proj.GetAttrValue("AUTHORITY", 1):
                out_log_str += "  Error appending to existing NetCDF '{}': different EPSG.".format(roi_save_name) + '\n'
                return [out_log_str, out_cpu_arr, out_nc_flag]
            if not getattr(roi_nc_grp,'LakeType')=="undefined":   # clear/deep,turbid/shallow ... should be defined somehow!
                out_log_str += "  Error appending to existing NetCDF '{}': different lake type.".format(roi_save_name) + '\n'
                return [out_log_str, out_cpu_arr, out_nc_flag]

            tmp = getattr(roi_nc_grp,'GAR_thresholds')
            c1 = np.size(tmp)!=np.size(alert_thresholds)
            c2 = (np.size(tmp)==1 and (~np.isnan(tmp) | ~np.isnan(alert_thresholds)))
            c3 = (np.size(tmp)==2 & ~(tmp==alert_thresholds).all())
            if c1 or c2 or c3:
                out_log_str += "  Error appending to existing NetCDF '{}': different GAR thresholds.".format(roi_save_name) + '\n'
                return [out_log_str, out_cpu_arr, out_nc_flag]

            if len( roi_nc_dates )==0:
                out_log_str += "  Error: existing dataset '{}' has 0 dates.".format(roi_save_name) + '\n'
                return [out_log_str, out_cpu_arr, out_nc_flag]

            # get start date from existing .nc file:
            tmp = getattr(roi_nc_grp,'LastProcDate')   # avoids re-processing no-data time window / dates
            roi_nc_last_date = datetime(int(tmp[:4]), int(tmp[5:7]), int(tmp[8:10]) )
            roi_start_date = roi_nc_last_date + timedelta(days=1)

            roi_nc_grp.close()
            out_log_str += "  Found existing NetCDF file '{}'; will append to it.".format(roi_save_name) + '\n'
            out_log_str += "    Last processed date of existing dataset: " + roi_nc_last_date.strftime('%Y-%m-%d') + '\n'
        
        else:   # no existing file: cannot append
            tmp = "  Could not find existing NetCDF file '{}' to append to; will create new file.".format(roi_save_name)
            out_log_str += tmp + '\n'
        #---end: if roi_file_exists
            
        # check for existing 'no-data' log file: if found, load LastProcDate
        if nodata_file_exists:   # polygon has been processed previously, check LastProcDate
            out_log_str += "  Found existing \"no-data\" file '{}'.".format(nodata_log_name) + '\n'

            # read contents:
            dic = {}
            with open(nodata_log_name,"r") as ndf:
                for line in ndf:
                   (key, val) = line[:-1].split(None,1)
                   dic[key] = val

            tmp = (not dic.has_key('Name') or dic['Name']!=lakes_name_list[ftr])
            tmp = tmp or (not dic.has_key('DisplayName') or dic['DisplayName']!=lakes_dispname_list[ftr])
            tmp = tmp or (not dic.has_key('ResultsVersion') or dic['ResultsVersion']!=results_version)
            tmp = tmp or not dic.has_key('LastProcDate')
            if tmp:
                out_log_str += "    Inconsistent \"no-data\" file; will be ignored." + '\n'
            else:
                tmp = dic['LastProcDate']   # avoids re-processing no-data time window / dates
                nodata_last_date = datetime(int(tmp[:4]), int(tmp[5:7]), int(tmp[8:10]) )
                nodata_start_date = nodata_last_date + timedelta(days=1)
                out_log_str += "    Last previously processed date: " + nodata_last_date.strftime('%Y-%m-%d') + '\n'
                
                if nodata_start_date>roi_start_date or not roi_append_nc:
                    roi_start_date = nodata_start_date
            #---end: if tmp
        #---end: if nodata_file_exists
    #---end: if append_to_existing_nc

    if end_date==None:
        tmp = datetime.today()
        roi_end_date = datetime(tmp.year,tmp.month,tmp.day)
    else:
        roi_end_date = datetime( int(end_date[:4]), int(end_date[5:7]), int(end_date[8:10]) )
    roi_end_date = roi_end_date + timedelta(days=1)

    tmp = "  Date range for processing: " + roi_start_date.strftime('%Y-%m-%d') + " (inclusive) to " 
    tmp = tmp + roi_end_date.strftime('%Y-%m-%d') + " (exclusive)"
    out_log_str += tmp + '\n'

    if roi_start_date >= roi_end_date:
        out_log_str += "  No data to process: start date of time window is on or after end date." + '\n'
        return [out_log_str, out_cpu_arr, out_nc_flag]


    ###########################################################################
    ### Iteration over polygons: load and process AGDC time series
    ###########################################################################
    
    # AGDC defs:
    dimensions = { 'lon' : { 'range': (min_lon, max_lon) },
                   'lat' : { 'range': (max_lat, min_lat) },
                   'time': { 'range': ( (roi_start_date.year, roi_start_date.month, roi_start_date.day), 
                                        (roi_end_date.year, roi_end_date.month, roi_end_date.day) ) } }

    # Cycling through satellites:
    n_sats = len(WQ_sats)
    proc_init = True
    wofs_init = True
    wofs_error = False
    concat_init = True
    n_dates_orig = 0
    last_proc_date = datetime(1,1,1)
    
    dc_api = datacube.api.API()
    ae = AnalyticsEngine(api=dc_api)
    ee = ExecutionEngine(api=dc_api)

    for sat in range(n_sats):

        ### Check for satellite availability:
        sat_defs = WQ_sats.values()[sat]
        sat_str = sat_defs['API_sat_str']
        plat_str = sat_defs['API_plat_str']
        out_log_str += "  Processing {} data (satellite {} of {}) ... ".format(plat_str,sat+1,n_sats) + '\n'

        if sat_str not in list(dc_prods['name']):
            out_log_str += "    Satellite data '{}' not available in current database.".format(sat_str) + '\n'
            continue   # next satellite
        

        ### Check for data availability:
        query = { 'platform': plat_str,
                  'product': nbar_prod,
                  'dimensions': dimensions }
        try:
            tmp = dc_api.get_descriptor(query, include_storage_units=False)
            if len( tmp[sat_str]['irregular_indices']['time'] )==0:
                out_log_str += "    No '{}' satellite data available in current database, for the selected time window and ROI.".format(sat_str) + '\n'
                continue
        except:
            out_log_str += "    No '{}' satellite data available in current database, for the selected time window and ROI.".format(sat_str) + '\n'
            continue
        
        
        ### Load data using AE/EE:
        ae.plan = list()   # appears to be "resetting" the AE definitions...
        # ae = AnalyticsEngine()
        # ee = ExecutionEngine()
        
        ae_green = ae.create_array( (plat_str, nbar_prod), [gbnd], dimensions, 'ae_green' )
        ae_red   = ae.create_array( (plat_str, nbar_prod), [rbnd], dimensions, 'ae_red' )
        ae_swir2 = ae.create_array( (plat_str, nbar_prod), [s2bnd], dimensions, 'ae_swir2' )
        aePQ     = ae.create_array( (plat_str, pq_prod), [pqbnd], dimensions, 'aePQ' )

        # selected WQ algorithm:
        if WQ_type=='Alg1:(red+green)/2':
            aeWQ = ae.apply_expression( [ae_green, ae_red], '((array1 + array2) * 0.5)', 'aeWQ' )
        elif WQ_type=='Alg2:TwoBand_TSS_calib':
            cfac = sat_defs['Alg2_calib_fac']
            cexp = sat_defs['Alg2_calib_exp']
            # tmp = '( {} * ((array1 + array2)*0.00005)**{} )'.format(cfac,cexp)   # logical formulation, doesn't work!
            tmp = '{}*exp(log((array1+array2)*0.00005)*{})'.format(cfac,cexp)   # hack to implement the power function...
            aeWQ = ae.apply_expression( [ae_green, ae_red], tmp, 'aeWQ' )
        else:
            raise NotImplementedError( "Water quality algorithm '{}' not implemented.".format(WQ_type) )

        ee.execute_plan(ae.plan)
        WQ_xarray = ee.cache['aeWQ']['array_result']['aeWQ']
        PQ_xarray = ee.cache['aePQ']['array_result'][pqbnd]
        S2_xarray = ee.cache['ae_swir2']['array_result'][s2bnd]
        
        WQ_xarray = WQ_xarray.load()   # allows about 8-fold speed-up by avoiding multiple lazy (re-)loading...
        PQ_xarray = PQ_xarray.load()
        S2_xarray = S2_xarray.load()

        # convert UTC dates to solar days:
        for ii in range(WQ_xarray.shape[0]):
            WQ_xarray['time'].values[ii] = solar_day(WQ_xarray['time'].values[ii], mean_lon).astype('datetime64[ns]')
        for ii in range(PQ_xarray.shape[0]):
            PQ_xarray['time'].values[ii] = solar_day(PQ_xarray['time'].values[ii], mean_lon).astype('datetime64[ns]')
        for ii in range(S2_xarray.shape[0]):
            S2_xarray['time'].values[ii] = solar_day(S2_xarray['time'].values[ii], mean_lon).astype('datetime64[ns]')
        
        n_dates = WQ_xarray.shape[0]
        n_dates_orig = n_dates_orig + n_dates
        tmp1 = WQ_xarray['time'].values
        tmp = "    {} time slices were found for the selected time window and ROI".format(n_dates)
        tmp = tmp + " ({} to {}).".format( str(tmp1[0])[:10], str(tmp1[-1])[:10] )
        out_log_str += tmp + '\n'
        
        tmp = datetime( int(str(tmp1[-1])[:4]), int(str(tmp1[-1])[5:7]), int(str(tmp1[-1])[8:10]) )
        if last_proc_date<tmp:
            last_proc_date = tmp
        
        
        ### Initialise proj. and poly. variables: once-only operations
        if proc_init:
            proc_init = False

            # Satellite projection data: assume all satellites have same projection!
            tmp = dc_prods[dc_prods.name==sat_str]['crs'].item().wkt
            sat_proj = osr.SpatialReference( wkt=tmp )

            sat_xvec = WQ_xarray['x'].values
            sat_yvec = WQ_xarray['y'].values
            npix_x = len(sat_xvec)
            npix_y = len(sat_yvec)

            sat_pix_size = sat_xvec[1] - sat_xvec[0]   # assume uniform square pixels
            sat_geotx = ( sat_xvec[0], sat_pix_size, 0, sat_yvec[0], 0, -sat_pix_size )

            # Polygon mask:
            geo_to_sat_proj_tx = osr.CoordinateTransformation( geo_proj, sat_proj )
            tmp = geo_to_sat_proj_tx.TransformPoints( poly_array_geo )
            poly_array_xy = np.array( tmp )[:,:2]
            polyPath = Path(poly_array_xy)
            poly_mask_xy = np.ones(WQ_xarray.shape[1:], dtype='bool')

            jj1 = np.where( sat_xvec<min(poly_array_xy[:,0]) )[0][-1]
            jj2 = np.where( sat_xvec>max(poly_array_xy[:,0]) )[0][0]
            ii1 = np.where( sat_yvec>max(poly_array_xy[:,1]) )[0][-1]
            ii2 = np.where( sat_yvec<min(poly_array_xy[:,1]) )[0][0]
            njj = jj2 - jj1 + 1

            for ii in range( ii1, ii2+1 ):
                tmp = np.stack( (sat_xvec[jj1:jj2+1], sat_yvec[ii]*np.ones(njj)), axis=1 )
                poly_mask_xy[ii,jj1:jj2+1] = ~polyPath.contains_points( tmp )

            n_pix_poly = np.sum( ~poly_mask_xy )
            n_valid_pix_min = lake_min_valid_prc * n_pix_poly / 100.0

            # Buffered polygon mask:
            poly_mask_xy_bufd = poly_mask_xy.copy()
            poly_mask_xy_bufd = expand_mask( poly_mask_xy_bufd, polygon_buffer_width )
        #---end: if proc_init


        ### Process time series: apply non-buffered-polygon + PQ masking to WQ and SWIR2 data
        keep_ind = np.zeros(n_dates).astype('bool')    # keep track of which dates (if any) to remove
        PQ_dates = PQ_xarray['time'].values

        for dd in range(n_dates):
            cur_date = WQ_xarray['time'].values[dd]    # current date
            idx = np.where(PQ_dates==cur_date)[0]
            if len(idx)==0:    
                continue   # time slice not found in PQ dataset: remove WQ slice (unable to mask properly)

            # time slice found in PQ dataset: mask and keep date in time series
            keep_ind[dd] = True
            tmp = PQ_xarray[idx[0]].values.astype(int)
            msk = ( (tmp & cloud_bit_mask)!=cloud_bit_mask ).astype(bool)
            msk = expand_mask( msk, cloud_buffer_width )    # expanded cloud mask
            msk = msk | ( (tmp & other_bit_mask)!=other_bit_mask ).astype(bool)   # saturation, sea, contiguity
            msk = msk | poly_mask_xy   # non-buffered-polygon mask
            WQ_xarray.values[dd][msk] = np.nan
            S2_xarray.values[dd][msk] = np.nan
        #---end: for dd in range(n_dates)

        tmp = np.sum(keep_ind)
        if n_dates!=tmp:
            out_log_str += "    {} time slices were removed due to discrepancies with the PQ dataset.".format( n_dates-tmp) + '\n'
        if tmp==0:
            out_log_str += "    --> No data left for this satellite." + '\n'
            continue
        WQ_xarray = WQ_xarray[keep_ind]   # remove unwanted time slices
        S2_xarray = S2_xarray[keep_ind]


        ### Merge dates: this should theoretically process WQ and SWIR2 data identically...
        WQ_xarray = merge_dup_dates( WQ_xarray.load() )
        S2_xarray = merge_dup_dates( S2_xarray.load() )
        out_log_str += "    {} time slices left after merging duplicated dates.".format(WQ_xarray.shape[0]) + '\n'


        ### Remove "empty" time slices (ie. nr of valid pixels below threshold):
        tmp = ( ~np.isnan(WQ_xarray) ).sum('x').sum('y').values
        idx = np.where( tmp>=n_valid_pix_min )[0]
        n_dates = len(idx)
        out_log_str += "    {} time slices left following PQ & polygon masking.".format(n_dates) + '\n'
        if n_dates==0:
            out_log_str += "    --> No data left for this satellite." + '\n'
            continue
        WQ_xarray = WQ_xarray[idx]
        S2_xarray = S2_xarray[idx]


        ### Initialise WOFS variables: once-only (only if some data available / left)
        if wofs_init:
            wofs_init = False

            # load WOFS data and reproject to UTM / satellite projection:
            try:
                wofs_array, wofs_proj, wofs_geotx, tmp1, tmp2 = load_wofs_data(
                                                wofs_dir_path, min_lon, max_lon, min_lat, max_lat )
            except RuntimeError as err:
                out_log_str += "  Error loading WOFS data: {}".format(err) + '\n'
                wofs_error = True
                break

            # WOFS gdal dataset:
            nr, nc = wofs_array.shape
            gdal_data = gdal.GetDriverByName( 'MEM' ).Create('', nc, nr, 1, gdal.GDT_Float32)
            gdal_data.SetGeoTransform( wofs_geotx )
            gdal_data.SetProjection( wofs_proj )
            gdal_data.GetRasterBand(1).WriteArray( wofs_array, 0, 0)

            # reprojected WOFS gdal dataset:
            gdal_proj_data = gdal.GetDriverByName( 'MEM' ).Create('', len(sat_xvec), len(sat_yvec), 1, gdal.GDT_Float32)
            gdal_proj_data.SetGeoTransform( sat_geotx )
            gdal_proj_data.SetProjection( sat_proj.ExportToWkt() )
            gdal_proj_data.GetRasterBand(1).WriteArray( np.zeros((len(sat_yvec),len(sat_xvec))), 0, 0) # prefill with 0
            tmp = gdal.ReprojectImage( gdal_data, gdal_proj_data,
                                       wofs_proj, sat_proj.ExportToWkt(),
                                       gdal.GRA_Bilinear )    # gdal.GRA_NearestNeighbour by default

            # WOFS mask, also masked by polygon (both non-buffered):
            wofs_mask_xy = gdal_proj_data.ReadAsArray()
            wofs_mask_xy = (wofs_mask_xy<wofs_prc_thr)
            wofs_mask_xy = wofs_mask_xy | poly_mask_xy
            n_pix_wofs = (~wofs_mask_xy).sum()
        #---end: if wofs_init


        ### Process time series: calculate WOFS and poly stats / flags, apply SWIR2 filter
        low_water_flag_list = ["undefined" for ii in range(n_dates)]
        wofs_flag_list = ["ephemeral" for ii in range(n_dates)]

        for dd in range(n_dates):
            WQ_arr = WQ_xarray[dd].values   # poly & PQ masked

            # polygon stats:
            tmp = np.sum( ~np.isnan(WQ_arr) )
            n_pix_poly_nan = n_pix_poly - tmp   # low-water stats

            # WOFS masking & stats (for stats only, not applied to WQ):
            if n_pix_wofs!=0:
                wq_tmp = WQ_arr.copy()
                wq_tmp[wofs_mask_xy] = np.nan
                tmp = np.sum( ~np.isnan(wq_tmp) )
                n_pix_wofs_nan = n_pix_wofs - tmp   # WOFS stats

            # apply SWIR2 mask, calculate water stats:
            msk = S2_xarray[dd].values
            msk = (msk>(SWIR2_prc_thr*100.0))   # SWIR2_prc_thr * 10000.0 / 100.0
            WQ_xarray.values[dd][msk] = np.nan
            n_water_pix_swir = np.sum( ~np.isnan(WQ_xarray[dd].values) )

            # low-water flag:
            nan_pix_prct = 100.0*n_pix_poly_nan/n_pix_poly
            water_pix_prct = 100.0*n_water_pix_swir/n_pix_poly
            if nan_pix_prct<=(100.0-low_water_prc_thr):
                if water_pix_prct<(low_water_prc_thr-nan_pix_prct):
                    low_water_flag = 'true'
                elif water_pix_prct>=low_water_prc_thr:
                    low_water_flag_list[dd] = 'false'
            #   else: low_water_flag_list[dd] = 'undefined'
            else:
                if water_pix_prct<(low_water_prc_thr-nan_pix_prct):
                    low_water_flag_list[dd] = 'true'
            #   else: low_water_flag_list[dd] = 'undefined'

            # WOFS water flag:
            if n_pix_wofs!=0:
                if n_water_pix_swir>(n_pix_wofs+n_pix_wofs_nan):
                    wofs_flag_list[dd] = "perennial: larger than WOFS {}% extents".format(wofs_prc_thr)
                elif n_water_pix_swir<(n_pix_wofs-n_pix_wofs_nan):
                    wofs_flag_list[dd] = "perennial: smaller than WOFS {}% extents".format(wofs_prc_thr)
                else:
                    wofs_flag_list[dd] = "perennial: undefined"
        #---end: for dd in range(n_dates)


        ### Remove "empty" time slices (ie. nr of valid pixels below threshold):
        tmp = ( ~np.isnan(WQ_xarray) ).sum('x').sum('y').values
        idx = np.where( tmp>=n_valid_pix_min )[0]
        n_dates = len(idx)
        out_log_str += "    {} time slices left following SWIR2 masking.".format(n_dates) + '\n'
        if n_dates==0:
            out_log_str += "    --> No data left for this satellite." + '\n'
            continue
        WQ_xarray = WQ_xarray[idx]
        low_water_flag_list = [low_water_flag_list[ii] for ii in idx]
        wofs_flag_list = [wofs_flag_list[ii] for ii in idx]


        ### Process time series: apply buffered-polygon to WQ
        for dd in range(n_dates): 
            WQ_xarray.values[dd][poly_mask_xy_bufd] = np.nan


        ### Remove "empty" time slices (ie. nr of valid pixels below threshold):
        tmp = ( ~np.isnan(WQ_xarray) ).sum('x').sum('y').values
        idx = np.where( tmp>=n_valid_pix_min )[0]
        n_dates = len(idx)
        out_log_str += "    {} time slices left following buffered polygon masking.".format(n_dates) + '\n'
        if n_dates==0:
            out_log_str += "    --> No data left for this satellite." + '\n'
            continue
        WQ_xarray = WQ_xarray[idx]
        low_water_flag_list = [low_water_flag_list[ii] for ii in idx]
        wofs_flag_list = [wofs_flag_list[ii] for ii in idx]


        ### Concatenate dataset with overall time series:
        sat_flag_list = [plat_str for ii in range(n_dates)]
        if concat_init:
            # initialise variables
            concat_init = False
            WQ_xarray_TS = WQ_xarray.copy()
            sat_flag_list_TS = sat_flag_list[:]   # list copy
            low_water_flag_list_TS = low_water_flag_list[:]
            wofs_flag_list_TS = wofs_flag_list[:]
        else:
            # concatenate dates
            concat_dates = np.concatenate( (WQ_xarray_TS['time'].values, WQ_xarray['time'].values) )
            sort_res = np.argsort(concat_dates)

            # concatenate flags
            tmp = sat_flag_list_TS + sat_flag_list   # list concatenation
            sat_flag_list_TS = [tmp[ii] for ii in sort_res]
            tmp = low_water_flag_list_TS + low_water_flag_list
            low_water_flag_list_TS = [tmp[ii] for ii in sort_res]
            tmp = wofs_flag_list_TS + wofs_flag_list
            wofs_flag_list_TS = [tmp[ii] for ii in sort_res]

            # concatenate data
            sat1_idx = np.array( [ii for ii,ss in enumerate(sat_flag_list_TS) if ss!=plat_str] )
            sat2_idx = np.array( [ii for ii,ss in enumerate(sat_flag_list_TS) if ss==plat_str] )
            WQ_xarray_TS = xr.concat( [WQ_xarray_TS, WQ_xarray], dim='time', positions=[sat1_idx,sat2_idx] )
        #---end: if concat_init
    #---end: for sat in range(n_sats)
    
#     dc_api.datacube.index.close()
    dc_api.datacube.close()
    
    if wofs_error:   # if error processing WOFS data, continue to next polygon
        return [out_log_str, out_cpu_arr, out_nc_flag]

    if concat_init:   # no data available/left for this polygon / ROI
        out_log_str += "  There is no valid data available for this polygon/ROI." + '\n'
        
        if not last_proc_date==datetime(1,1,1):
            with open(nodata_log_name,"w") as logf:   # print info to a "no-data" log file
                logf.write( "Name {}\n".format(lakes_name_list[ftr]) )
                logf.write( "DisplayName {}\n".format(lakes_dispname_list[ftr]) )
                logf.write( "ResultsVersion {}\n".format(results_version) )
                logf.write( "LastProcDate {}\n".format(str(last_proc_date)[:10]) )  # dataset processed up to that date
            out_log_str += "  Saved ROI's last processed date to log file '{}'.".format(nodata_log_name) + '\n'
            
        out_log_str += "  (Polygon area is {} by {} pixels)".format(npix_x,npix_y) + '\n'
        return [out_log_str, out_cpu_arr, out_nc_flag]

    n_dates = WQ_xarray_TS.shape[0]
    out_log_str += "  There are {} time slices in the resulting multi-sensor time series.".format(n_dates) + '\n'


    ###########################################################################
    ### Iteration over polygons: reprojecting to geodetic
    ###########################################################################

    # Reproject each time slice to geodetic:
    npix_lon = len(lon_vec)
    npix_lat = len(lat_vec)
    n_dates, npix_y, npix_x = WQ_xarray_TS.shape
    WQ_array_TS_geo = np.ones( (n_dates, npix_lat, npix_lon) ) * np.nan

    for dd in range(n_dates):
        # WQ gdal dataset:
        gdal_data = gdal.GetDriverByName( 'MEM' ).Create('', npix_x, npix_y, 1, gdal.GDT_Float32)
        gdal_data.SetGeoTransform( sat_geotx )
        gdal_data.SetProjection( sat_proj.ExportToWkt() )
        gdal_data.GetRasterBand(1).WriteArray( WQ_xarray_TS[dd].values, 0, 0)

        # reprojected WQ gdal dataset:
        gdal_proj_data = gdal.GetDriverByName( 'MEM' ).Create('', npix_lon, npix_lat, 1, gdal.GDT_Float32)
        gdal_proj_data.SetGeoTransform( (lon_vec[0], pix_size_geo, 0, lat_vec[-1], 0, -pix_size_geo) )
        gdal_proj_data.SetProjection( geo_proj.ExportToWkt() )

        gdal_proj_data.GetRasterBand(1).WriteArray( np.ones((npix_lat,npix_lon))*np.nan, 0, 0)
        tmp = gdal.ReprojectImage( gdal_data, gdal_proj_data,
                                   sat_proj.ExportToWkt(), geo_proj.ExportToWkt(),
                                   gdal.GRA_NearestNeighbour)
                                   # gdal.GRA_Bilinear )   # found to eat away at edges of lake boundaries!...
        WQ_array_TS_geo[dd] = gdal_proj_data.ReadAsArray()
    #---end: for dd in range(n_dates)


    ###########################################################################
    ### Iterating over polygons: Lake stats & Green-Amber-Red flags
    ###########################################################################

    mean_vec = np.nanmean( WQ_array_TS_geo, (1,2) )
    med_vec = np.nanmedian( WQ_array_TS_geo, (1,2) )
    max_vec = np.nanmax( WQ_array_TS_geo, (1,2) )
    min_vec = np.nanmin( WQ_array_TS_geo, (1,2) )

    if np.size(alert_thresholds)==1:   # np.nan
        GAR_flag_list = ['undefined' for ii in range(n_dates)]
    else:
        amb_idx = np.where( np.nansum( WQ_array_TS_geo>alert_thresholds[0], (1,2) )>0 )[0]
        red_idx = np.where( np.nansum( WQ_array_TS_geo>alert_thresholds[1], (1,2) )>0 )[0]
        GAR_flag_list = ['green' for ii in range(n_dates)]
        for ii in amb_idx: GAR_flag_list[ii] = "amber"
        for ii in red_idx: GAR_flag_list[ii] = "red"


    ###########################################################################
    ### Iterating over polygons: CPU info
    ###########################################################################

    poly_t1 = timer.time()
    delta_t = poly_t1 - poly_t0

    lake_area_ha = shape_list[ftr]/10000.0
    box_area_ha = len(sat_xvec) * len(sat_yvec) * (sat_pix_size**2) / 10000.0
    window_days = (roi_end_date - roi_start_date).days

    # saving CPU data for later analysis:
    out_cpu_arr[0,0] = delta_t
    out_cpu_arr[0,1] = lake_area_ha
    out_cpu_arr[0,2] = box_area_ha
    out_cpu_arr[0,3] = window_days
    out_cpu_arr[0,4] = n_dates_orig
    out_cpu_arr[0,5] = n_dates


    ###########################################################################
    ### Iteration over polygons: saving to NetCDF
    ###########################################################################
    
    # Extracting dates in datetime format:
    dt_dates = np.zeros(n_dates).astype(datetime)
    for ii in range(n_dates):
        tmp = str( WQ_xarray_TS['time'].values[ii] )
        dt_dates[ii] = datetime( int(tmp[0:4]), int(tmp[5:7]), int(tmp[8:10]), int(tmp[11:13]), 
                                 int(tmp[14:16]), int(tmp[17:19]), int(tmp[20:26]) )
    dt_dates = date2num(dt_dates, units=netcdf_time_units, calendar=netcdf_time_calendar )

    if roi_append_nc:   # append to existing NetCDF file

        roi_nc_grp = Dataset(roi_save_name, mode='a')
        n_ex_dates = roi_nc_grp.dimensions['time'].size   # nr of existing dates
        idx = np.arange(n_ex_dates, n_ex_dates+n_dates)

        # load and append to variables:
        wq_vals = roi_nc_grp.variables['WQ_data']
        wq_vals[idx,:,:] = WQ_array_TS_geo

        times = roi_nc_grp.variables['time']
        times[idx] = dt_dates

        mean_vals = roi_nc_grp.variables['WQ_means']
        mean_vals[idx] = mean_vec

        med_vals = roi_nc_grp.variables['WQ_medians']
        med_vals[idx] = med_vec

        max_vals = roi_nc_grp.variables['WQ_maxvals']
        max_vals[idx] = max_vec

        min_vals = roi_nc_grp.variables['WQ_minvals']
        min_vals[idx] = min_vec

        sat_vals = roi_nc_grp.variables['satellite_flags']
        sat_vals[idx] = np.array(sat_flag_list_TS)

        water_vals = roi_nc_grp.variables['low_water_flags']
        water_vals[idx] = np.array(low_water_flag_list_TS)

        wofs_vals = roi_nc_grp.variables['wofs_flags']
        wofs_vals[idx] = np.array(wofs_flag_list_TS)

        gar_vals = roi_nc_grp.variables['GAR_flags']
        gar_vals[idx] = np.array(GAR_flag_list)

        roi_nc_grp.LastProcDate = str(last_proc_date)[:10]  # dataset processed up to that date
        roi_nc_grp.DateLastUpdated = datetime.now().strftime('%Y/%m/%d %H:%M:%S')
        roi_nc_grp.LastUpdateCPUinfo = (delta_t, lake_area_ha, box_area_ha, window_days, n_dates_orig, n_dates)

        roi_nc_grp.close()
        out_log_str += "  WQ and ancillary data appended to file '{}'.".format(roi_save_name) + '\n'
        
    else:   # create new or replace existing NetCDF file

        # backup the existing NetCDF file if desired
        if roi_file_exists & backup_if_replacing_nc:
            # create backup directory:
            bak_dir_path = save_dir_path + 'backup/'
            if not os.path.exists( bak_dir_path ):
                os.makedirs( bak_dir_path )
                if not os.path.exists( bak_dir_path ):
                    out_log_str += "  Error: could not create backup directory '{}'.".format(bak_dir_path) + '\n'
                    return [out_log_str, out_cpu_arr, out_nc_flag]
                out_log_str += "  Created backup directory '{}'.".format(bak_dir_path) + '\n'

            # create backup:
            tmp = bak_dir_path+roi_file_str
            os.rename(roi_save_name, tmp)
            if not os.path.isfile( tmp ):
                out_log_str += "  Error: could not back up NetCDF file '{}'.".format(tmp) + '\n'
                return [out_log_str, out_cpu_arr, out_nc_flag]
            out_log_str += "  Existing NetCDF file backed up to '{}'.".format(tmp) + '\n'
        #---end if

        # open NetCDF dataset:
        roi_nc_grp = Dataset(roi_save_name, mode='w')   # overwrites existing file

        # create NetCDF dimensions
        time = roi_nc_grp.createDimension("time", None)   # unlimited dimension
        lat = roi_nc_grp.createDimension("lat", len(lat_vec) )
        lon = roi_nc_grp.createDimension("lon", len(lon_vec) )

        # create NetCDF variables
        times = roi_nc_grp.createVariable("time","f8",("time",))
        lats = roi_nc_grp.createVariable("lat","f8",("lat",))
        lons = roi_nc_grp.createVariable("lon","f8",("lon",))

        wq_vals = roi_nc_grp.createVariable("WQ_data","f8",("time","lat","lon",))
        mean_vals = roi_nc_grp.createVariable("WQ_means","f8",("time",))
        med_vals = roi_nc_grp.createVariable("WQ_medians","f8",("time",))
        max_vals = roi_nc_grp.createVariable("WQ_maxvals","f8",("time",))
        min_vals = roi_nc_grp.createVariable("WQ_minvals","f8",("time",))

        sat_vals = roi_nc_grp.createVariable("satellite_flags","S32",("time",))   # Define str length long enough for flags!
        water_vals = roi_nc_grp.createVariable("low_water_flags","S16",("time",))   # Define str length long enough for flags!
        wofs_vals = roi_nc_grp.createVariable("wofs_flags","S64",("time",))   # Define str length long enough for flags!
        gar_vals = roi_nc_grp.createVariable("GAR_flags","S8",("time",))   # Define str length long enough for flags!

        # assign values to variables
        times.units = netcdf_time_units
        times.calendar = netcdf_time_calendar
        times[:] = dt_dates
        lons[:] = lon_vec
        lats[:] = lat_vec[::-1]

        wq_vals[:,:,:] = WQ_array_TS_geo
        mean_vals[:] = mean_vec
        med_vals[:] = med_vec
        max_vals[:] = max_vec
        min_vals[:] = min_vec

        sat_vals[:] = np.array(sat_flag_list_TS)
        water_vals[:] = np.array(low_water_flag_list_TS)
        wofs_vals[:] = np.array(wofs_flag_list_TS)
        gar_vals[:] = np.array(GAR_flag_list)

        # dataset attributes:
        roi_nc_grp.Name = lakes_name_list[ftr]
        roi_nc_grp.DisplayName = lakes_dispname_list[ftr]
        roi_nc_grp.EPSG = geo_proj.GetAttrValue("AUTHORITY", 1)
        roi_nc_grp.LakeType = "undefined"   # clear/deep vs. turbid/shallow ... should be defined somehow!
        roi_nc_grp.WQtype = WQ_type
        roi_nc_grp.GAR_thresholds = alert_thresholds
        roi_nc_grp.LastProcDate = str(last_proc_date)[:10]  # dataset processed up to that date
        roi_nc_grp.DateCreated = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        roi_nc_grp.DateLastUpdated = "NA"
        roi_nc_grp.LastUpdateCPUinfo = (delta_t, lake_area_ha, box_area_ha, window_days, n_dates_orig, n_dates)
        roi_nc_grp.ResultsVersion = results_version

        roi_nc_grp.close()

        if not os.path.isfile( roi_save_name ):
            out_log_str += "  Error: could not create NetCDF file '{}'.".format(roi_save_name) + '\n'
            return [out_log_str, out_cpu_arr, out_nc_flag]
        out_log_str += "  WQ and ancillary data saved to file '{}'.".format(roi_save_name) + '\n'
    #---end if append_to_existing_nc
    
    out_nc_flag = True
    
    if nodata_file_exists:
        os.remove(nodata_log_name)
        out_log_str += "  Deleted \"no-data\" file '{}'.".format(nodata_log_name) + '\n'

    # Log info:
    tmp = delta_t / lake_area_ha
    out_log_str += "  Total time processing polygon (h:m:s): {}".format(str(timedelta(seconds=delta_t))) + '\n'
    out_log_str += "  (Polygon area is {} by {} pixels)".format(npix_x,npix_y) + '\n'
    
    return [out_log_str, out_cpu_arr, out_nc_flag]
#---end: def ftr_loop(ftr)

## User inputs

In [4]:
lakes_shp_file = '../NSW_lakes/NSW_WaterBody.shp'
   # shape file containing polygons for waterbodies of interest.
lake_min_area = 5000
   # waterbody screening: minimum area size in sq.m. for a waterbody to be considered.
lake_screen_perennial = True
   # waterbody screening flag: keep only perennial waterbodies?
polygon_buffer_width = 1.5
   # buffering of lake edges: distance (in Landsat pixels) from lake edges where pixels are masked out
lake_min_valid_prc = 10.0
   # minimum percentage of valid (non-NaN) extent required for lake to be processed / saved

pix_size_geo = 0.00025   
   # reprojection to geodetic: desired geodetic pixel size
lake_buffer_width_geo = 0.002  
   # buffer width to add around lake polygon extents for processing, in geodetic units

cloud_buffer_width = 12.0
   # buffer width (in Landsat pixels) to add around cloud and cloud shadow pixels in cloud mask

results_version = 'v3.1'
   # metadata version nr of current WQ outputs
save_dir_path = "/g/data/jr4/vis_data_v3.1/"
   # base directory path where the WQ outputs are saved: will be created if necessary; can contain existing data.
append_to_existing_nc = False  #True
   # flag: append to dataset on file if exists; otherwise, overwrite it (i.e. create new file).
backup_if_replacing_nc = True
   # flag: if existing dataset is overwritten, create backup first.
    
start_date = '2015-01-01'   # '2006-01-01'
   # start date of time window, format YYYY-MM-DD; will be determined from .nc file if exists and if appending
end_date = None   # '2015-12-31'  # '2014-01-01'   # None
   # end date, same format: if set to None, will use current date as end date; can be same as 'start_date'

wofs_dir_path = '/g/data2/fk4/wofs/current/pyramids/WaterSummary/0/'   
   # base path where geoTiff WOFS data is located
wofs_prc_thr = 90.0
   # percentage threshold for WOFS water mask

SWIR2_prc_thr = 1.0
   # percentage threshold for SWIR2 water mask
low_water_prc_thr = 50.0
   # percentage threshold below which the low-water flag is triggered (compared to polygon extents)

WQ_type = 'Alg2:TwoBand_TSS_calib'   # 'Alg1:(red+green)/2'
   # (string) type of water quality algorithm to use to generate results
alert_thresholds = None   # e.g. (2000,2400)
   # red/amber/green thresholds, defined as tuple (green_to_amber,amber_to_red); None if undefined
    
netcdf_time_units = "seconds since 1970-01-01 00:00:00.0"
   # time units to use when saving data to NetCDF format
netcdf_time_calendar = "standard"
   # calendar to use when saving data to NetCDF format

no_screen_outputs = True #False   #True
   # disables all printing to screen (e.g. when processing all 900 odd lakes)
num_threads = 12  #None
   # selected nr of threads for parallel processing; set to 'None' to set to all available CPUs

In [5]:
# Definition of satellites whose dataset will be used for WQ processing. These definitions need 
# to be determined on the basis of the current AGDC dataset (module) used.

# Landsat 8 definitions:
sat_1 = { 'API_sat_str' : 'ls8_nbar_albers',  # satellite string for API queries
          'API_plat_str' : 'LANDSAT_8',       # satellite platform string for API queries
          'Alg2_calib_fac' : 3957.0,          # calibration constant (multiplicative factor) for selected WQ algorithm
          'Alg2_calib_exp' : 1.6436 }         # calibration constant (exponent) for selected WQ algorithm

# Landsat 7 definitions:
sat_2 = { 'API_sat_str' : 'ls7_nbar_albers',
          'API_plat_str' : 'LANDSAT_7',
          'Alg2_calib_fac' : 3983.0,
          'Alg2_calib_exp' : 1.6246 }

# Landsat 5 definitions:
sat_3 = { 'API_sat_str' : 'ls5_nbar_albers',
          'API_plat_str' : 'LANDSAT_5',
          'Alg2_calib_fac' : 3983.0,
          'Alg2_calib_exp' : 1.6246 }

WQ_sats = { 'sat1' : sat_1,   # resulting dictionary of all satellite defs to use
            'sat2' : sat_2,
            'sat3' : sat_3 }

# API definitions, (currently) valid for all satellites:
API_defs = { 'nbar_product_str' : 'nbar',      # NBAR product string
             'red_band_str' : 'red',           # NBAR red band string
             'green_band_str' : 'green',       # NBAR green band string
             'swir2_band_str' : 'swir2',       # NBAR swir2 band string
             'pq_product_str' : 'pqa',         # PQ product string
             'pq_band_str' : 'pixelquality',   # PQ band string
             'green_satur_bit' : 1,        # PQ saturation bit for green band
             'red_satur_bit' : 2,          # PQ saturation bit for red band
             'swir2_satur_bit' : 7,        # PQ saturation bit for SWIR2 band
             'contig_bit' : 8,      # PQ contiguity bit 
             'land_bit' : 9,        # PQ land/sea bit
             'cloud_bit1' : 10,     # PQ cloud/shadow bits
             'cloud_bit2' : 11,
             'cloud_bit3' : 12,
             'cloud_bit4' : 13 }

## Miscellaneous

In [6]:
# Projection to use for final maps: here using geodetic, could be user-specific (would require some re-coding)
geo_proj = osr.SpatialReference ()
geo_proj.ImportFromEPSG ( 4326 );

In [7]:
if alert_thresholds==None:
    alert_thresholds = np.nan
else:
    alert_thresholds = np.array(alert_thresholds)
    if len(alert_thresholds)!=2 or alert_thresholds[0]>=alert_thresholds[1]:
        raise RuntimeError("Alert thresholds not defined properly (2 values in increasing order).")

In [8]:
# Create directories if necessary:
if not wofs_dir_path.endswith("/"):
    wofs_dir_path = wofs_dir_path + "/"
if not save_dir_path.endswith("/"):
    save_dir_path = save_dir_path + "/"
    
# Directory where some log info is saved for polygons without any valid data (avoids future re-processing)
nodata_dir_path = save_dir_path + "no_data/"

# log file:
log_base_path = save_dir_path + "WQproc_" + datetime.today().strftime("%Y-%m-%d_%H-%M-%S")

if not os.path.exists( save_dir_path ):
    os.makedirs( save_dir_path )
    if not os.path.exists( save_dir_path ):
        raise RuntimeError("Could not create directory '{}'.".format(save_dir_path) )
    log_file = open(log_base_path + ".log", "w")
    write_to_log( log_file, "Created results directory '{}'.".format(save_dir_path) )
else:
    log_file = open(log_base_path + ".log", "w")

if not os.path.exists( nodata_dir_path ):
    os.makedirs( nodata_dir_path )
    if not os.path.exists( nodata_dir_path ):
        raise RuntimeError("Could not create directory '{}'.".format(nodata_dir_path) )
    write_to_log( log_file, "Created \"no-data\" directory '{}'.".format(nodata_dir_path) )

In [9]:
# AGDC defs:
# dc_api = datacube.api.API()
dc_dc = datacube.Datacube()
# ae = AnalyticsEngine()
# ee = ExecutionEngine()
dc_prods = dc_dc.list_products()

pqbnd = API_defs['pq_band_str']
nbar_prod = API_defs['nbar_product_str']
pq_prod = API_defs['pq_product_str']
rbnd = API_defs['red_band_str']
gbnd = API_defs['green_band_str']
s2bnd = API_defs['swir2_band_str']

contig_bit = API_defs['contig_bit']
land_bit = API_defs['land_bit']
swir2_sat_bit = API_defs['swir2_satur_bit']
red_sat_bit = API_defs['red_satur_bit']
green_sat_bit = API_defs['green_satur_bit']
other_bit_mask = ( 1<<green_sat_bit | 1<<red_sat_bit | 1<<swir2_sat_bit | 1<<contig_bit | 1<<land_bit )

cloud_bit1 = API_defs['cloud_bit1']
cloud_bit2 = API_defs['cloud_bit2']
cloud_bit3 = API_defs['cloud_bit3']
cloud_bit4 = API_defs['cloud_bit4']
cloud_bit_mask = (1<<cloud_bit1 | 1<<cloud_bit2 | 1<<cloud_bit3 | 1<<cloud_bit4)

## Loading waterbody polygons from `.shp` file

In [10]:
# Open .shp file:
lakes_vec = ogr.Open( lakes_shp_file )
if lakes_vec==None:
    raise RuntimeError( "Cannot open shape file '{}'.".format(lakes_shp_file) )
if lakes_vec.GetLayerCount()!=1:
    raise RuntimeError( "File '{}' has 0 or multiple layers (1 expected).".format(lakes_shp_file) )
lakes_lyr = lakes_vec.GetLayer(0)

# List of waterbody names and attributes:
n_ftr = lakes_lyr.GetFeatureCount()
lname_list = list()
peren_list = list()
shape_list = list()
for kk in range( n_ftr ):
    ftr = lakes_lyr.GetFeature(kk)
    lname_list.append(ftr.GetField('hydroname'))   # name
    peren_list.append(ftr.GetField('perennia_1'))   # perennial flag
    shape_list.append(ftr.GetField('shape_STAr'))   # area in sq.m.

# Screen waterbodies of interest:
tmp = ( np.array(shape_list)>=lake_min_area )
if lake_screen_perennial: 
    tmp = ( (np.array(peren_list)=='Perennial') & tmp )
lakes_ftr_idx = np.where(tmp)[0]
n_lakes = len(lakes_ftr_idx)
if n_lakes==0:
    raise RuntimeError( "No waterbody left to process after screening.")
lakes_name_list = [ lname_list[ii] for ii in lakes_ftr_idx ]
shape_list = [ shape_list[ii] for ii in lakes_ftr_idx ]
lakes_dispname_list = lakes_name_list[:]   # list copy

# Check for duplicated names, rename them (append _0, _1, etc.):
renamed_cntr = 0
for kk in range(n_lakes):
    tmp = [ i for i,s in enumerate(lakes_name_list) if s==lakes_name_list[kk] ]
    n_tmp = len(tmp)
    if n_tmp>1:
        renamed_cntr = renamed_cntr + 1
        for ii,tt in enumerate(tmp):
            lakes_name_list[tt] = lakes_name_list[tt] + "_" + str(ii)

lakes_name_list = [ lakes_name_list[ii].replace(" ","_") for ii in range(n_lakes) ]

In [11]:
# Log info:
write_to_log( log_file, "Shape file '{}' info:".format(lakes_shp_file) )
write_to_log( log_file, "  There are {} waterbodies in the file '{}'.".format(n_ftr,lakes_shp_file) )
tmp = "  A total of {} {}".format(n_lakes,"perennial lakes" if lake_screen_perennial else "lakes")
tmp = tmp + "with area of more than {} sq.m. will be processed.".format(lake_min_area)
write_to_log( log_file, tmp )

if renamed_cntr>0:
    write_to_log( log_file, "  A total of {} lake names were re-labelled to avoid duplicates.".format(renamed_cntr) )

## Iteration over polygons ...

In [12]:
# Parallel processing of waterbodies:
n_cores = mp.cpu_count()
if num_threads==None:
    num_threads = n_cores

pool = mp.Pool(processes=num_threads)

poly_t_init = timer.time()

write_to_log( log_file, '\nCurrent system has {} available CPUs.'.format(n_cores) )
write_to_log( log_file, 'Starting parallel processing of {} polygons using {} threads...'.format(n_lakes,num_threads) )

ftr_idx = np.argsort(shape_list)[-1::-1]   # process larger lakes first (multi-processing)
res = pool.map(ftr_loop, ftr_idx)

In [13]:
# Process mp outputs:
n_nc_files = 0
cpu_data = np.ones((n_lakes,6))*np.nan
write_to_log( log_file, '' )

tmp = np.argsort(ftr_idx)
for ftr in tmp:
    write_to_log( log_file, res[ftr][0] )   # out_log_str
    cpu_data[ftr,:] = res[ftr][1]   # out_cpu_arr
    n_nc_files += res[ftr][2]   # out_nc_flag

In [14]:
delta_t = timer.time() - poly_t_init
tmp = str(timedelta(seconds=delta_t))
write_to_log( log_file, "\nTotal time processing all {} polygons (h:m:s): {}".format(n_lakes,tmp), True )
write_to_log( log_file, "A total of {} NetCDF files were created or updated.".format(n_nc_files), True )

## Assessment / analysis of computing times

In [15]:
rcp['axes.formatter.useoffset'] = False

reg = linear_model.LinearRegression()

idx = np.where( ~np.isnan(cpu_data[:,0]) )[0]
n_cpu = len(idx)

if n_cpu==0:
    write_to_log( log_file, "\nNo CPU times to analyse." )
else:
    cpu_data = cpu_data[idx,:]
    Y = np.log( cpu_data[:,0] )
    
    with PdfPages(log_base_path + ".pdf") as pdf:
        plt.figure(figsize=(7,7))
        plt.hist(Y, bins=60)
        plt.xlabel("log of proc. time (sec.)")
        plt.title( 'Proc. time histogram ({} polygons with available data)'.format(n_cpu) )
        pdf.savefig(); plt.close()

        plt.figure(figsize=(14,7)); plt.subplot(1,2,1)
        X = np.log(cpu_data[:,1]); plt.hist(X, bins=60)
        plt.xlabel("log of lake area (ha)"); plt.title('Lake area (polygon) histogram')
        plt.subplot(1,2,2)
        reg.fit( np.matrix(X).T, np.matrix(Y).T ); plt.scatter( X, Y, marker='.', edgecolor='blue', s=15 )
        xx = np.matrix( (np.nanmin(X), np.nanmax(X)) ).T; plt.plot( xx, reg.predict( xx ), 'k--' )
        plt.xlabel('log of lake area (ha)'); plt.ylabel('log of proc. time (sec.)');
        tmp = 'log_proc_time = {:.3g} * log_lake_area {:+.3g}'.format(reg.coef_[0][0],reg.intercept_[0])
        tmp = tmp + '\n(R^2 = {0:.3g})'.format(reg.score(np.matrix(X).T, np.matrix(Y).T))
        plt.title(tmp); pdf.savefig(); plt.close()

        plt.figure(figsize=(14,7)); plt.subplot(1,2,1)
        X = np.power( np.log(cpu_data[:,1]), 3 ); plt.hist(X, bins=60)
        plt.xlabel("log of lake area (ha) cubed"); plt.title('Lake area (polygon) histogram')
        plt.subplot(1,2,2)
        reg.fit( np.matrix(X).T, np.matrix(Y).T ); plt.scatter( X, Y, marker='.', edgecolor='blue', s=15 )
        xx = np.matrix( (np.nanmin(X), np.nanmax(X)) ).T; plt.plot( xx, reg.predict( xx ), 'k--' )
        plt.xlabel('log of lake area (ha) cubed'); plt.ylabel('log of proc. time (sec.)');
        tmp = 'log_proc_time = {:.3g} * log_lake_area^3 {:+.3g}'.format(reg.coef_[0][0],reg.intercept_[0])
        tmp = tmp + '\n(R^2 = {0:.3g})'.format(reg.score(np.matrix(X).T, np.matrix(Y).T))
        plt.title(tmp); pdf.savefig(); plt.close()

        plt.figure(figsize=(14,7)); plt.subplot(1,2,1)
        X = np.log(cpu_data[:,2]); plt.hist(X, bins=60)
        plt.xlabel("log of box area (ha)"); plt.title('Bounding box area (polygon extents) histogram')
        plt.subplot(1,2,2)
        reg.fit( np.matrix(X).T, np.matrix(Y).T ); plt.scatter( X, Y, marker='.', edgecolor='blue', s=15 )
        xx = np.matrix( (np.nanmin(X), np.nanmax(X)) ).T; plt.plot( xx, reg.predict( xx ), 'k--' )
        plt.xlabel('log of box area (ha)'); plt.ylabel('log of proc. time (sec.)');
        tmp = 'log_proc_time = {:.3g} * log_box_area {:+.3g}'.format(reg.coef_[0][0],reg.intercept_[0])
        tmp = tmp + '\n(R^2 = {0:.3g})'.format(reg.score(np.matrix(X).T, np.matrix(Y).T))
        plt.title(tmp); pdf.savefig(); plt.close()

        plt.figure(figsize=(14,7)); plt.subplot(1,2,1)
        X = np.power( np.log(cpu_data[:,2]), 3 ); plt.hist(X, bins=60)
        X_boxarea = X.copy()
        plt.xlabel("log of box area (ha) cubed"); plt.title('Bounding box area (polygon extents) histogram')
        plt.subplot(1,2,2)
        reg.fit( np.matrix(X).T, np.matrix(Y).T ); plt.scatter( X, Y, marker='.', edgecolor='blue', s=15 )
        xx = np.matrix( (np.nanmin(X), np.nanmax(X)) ).T; plt.plot( xx, reg.predict( xx ), 'k--' )
        plt.xlabel('log of box area (ha) cubed'); plt.ylabel('log of proc. time (sec.)');
        tmp = 'log_proc_time = {:.3g} * log_box_area^3 {:+.3g}'.format(reg.coef_[0][0],reg.intercept_[0])
        tmp = tmp + '\n(R^2 = {0:.3g})'.format(reg.score(np.matrix(X).T, np.matrix(Y).T))
        plt.title(tmp); pdf.savefig(); plt.close()

        plt.figure(figsize=(14,7)); plt.subplot(1,2,1)
        X = cpu_data[:,4]; plt.hist(X, bins=60)
        X_nrdates = X.copy()
        plt.xlabel("nr. of dates"); plt.title('Original nr. of time slices (all available) histogram')
        plt.subplot(1,2,2)
        reg.fit( np.matrix(X).T, np.matrix(Y).T ); plt.scatter( X, Y, marker='.', edgecolor='blue', s=15 )
        xx = np.matrix( (np.nanmin(X), np.nanmax(X)) ).T; plt.plot( xx, reg.predict( xx ), 'k--' )
        plt.xlabel('nr. of dates'); plt.ylabel('log of proc. time (sec.)');
        tmp = 'log_proc_time = {:.3g} * n_dates {:+.3g}'.format(reg.coef_[0][0],reg.intercept_[0])
        tmp = tmp + '\n(R^2 = {0:.3g})'.format(reg.score(np.matrix(X).T, np.matrix(Y).T))
        plt.title(tmp); pdf.savefig(); plt.close()

        plt.figure(figsize=(14,7)); plt.subplot(1,2,1)
        X = cpu_data[:,5]; plt.hist(X, bins=60)
        plt.xlabel("nr. of dates"); plt.title('Final nr. of time slices (final series) histogram')
        plt.subplot(1,2,2)
        reg.fit( np.matrix(X).T, np.matrix(Y).T ); plt.scatter( X, Y, marker='.', edgecolor='blue', s=15 )
        xx = np.matrix( (np.nanmin(X), np.nanmax(X)) ).T; plt.plot( xx, reg.predict( xx ), 'k--' )
        plt.xlabel('nr. of dates'); plt.ylabel('log of proc. time (sec.)');
        tmp = 'log_proc_time = {:.3g} * n_dates {:+.3g}'.format(reg.coef_[0][0],reg.intercept_[0])
        tmp = tmp + '\n(R^2 = {0:.3g})'.format(reg.score(np.matrix(X).T, np.matrix(Y).T))
        plt.title(tmp); pdf.savefig(); plt.close()

    write_to_log( log_file, "\nAssessment plots of CPU times saved to file '{}'.".format(log_base_path + ".pdf") )
    
    X_tmp = np.ones((len(X_boxarea),2))
    X_tmp[:,0] = X_boxarea
    X_tmp[:,1] = X_nrdates
    reg.fit( np.matrix(X_tmp), np.matrix(Y).T )
    tmp = '  Predicted proc. time per polygon:\n'
    tmp += '    log_proc_time = {:.3g} * log_box_area^3 {:+.3g} * n_dates_orig {:+.3g}  (R^2 = {:.3g})'
    tmp = tmp.format( reg.coef_[0][0], reg.coef_[0][1], reg.intercept_[0], reg.score(np.matrix(X_tmp),np.matrix(Y).T) )
    write_to_log( log_file, tmp )

    tmp = log_base_path + ".pckl"
    with open(tmp, 'wb') as ff:
        pickle.dump(cpu_data, ff)
    write_to_log( log_file, "CPU times saved to file '{}'.".format(tmp) )
#---end: if n_cpu==0

In [16]:
write_to_log( log_file, "\n+++ execution end +++\n")
log_file.close()